In [1]:
import os, re
from nltk.stem import PorterStemmer 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import treebank
from nltk.corpus import dependency_treebank
from nltk.grammar import DependencyGrammar
from nltk.parse import DependencyGraph
import pprint
from nltk.parse.corenlp import CoreNLPDependencyParser
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models import Word2Vec
import numpy as np
import re

In [2]:
ps = PorterStemmer()
wnl = WordNetLemmatizer()
dep_parser = CoreNLPDependencyParser(url='http://localhost:9000')

In [3]:
class Habitat:
    oid = -1
    name = ""

In [4]:
train_path = "./BioNLP-OST-2019_BB-norm_train/"
test_path = "./BioNLP-OST-2019_BB-norm_test/"
dev_path = "./BioNLP-OST-2019_BB-norm_dev/"
BB_train = os.listdir(train_path)
BB_test = os.listdir(test_path)
BB_dev = os.listdir(dev_path)
obp_file = "OntoBiotope_BioNLP-OST-2019.obo"

In [5]:
ontology = open(obp_file,encoding="utf8").read()

In [6]:
BB_train_a1 = sorted([name for name in BB_train if name.endswith("a1")])
BB_train_a2 = sorted([name for name in BB_train if name.endswith("a2")])
BB_train_txt = sorted([name for name in BB_train if name.endswith("txt")])

BB_dev_a1 = sorted([name for name in BB_dev if name.endswith("a1")])
BB_dev_a2 = sorted([name for name in BB_dev if name.endswith("a2")])
BB_dev_txt = sorted([name for name in BB_dev if name.endswith("txt")])

BB_test_a1 = sorted([name for name in BB_test if name.endswith("a1")])
BB_test_txt = sorted([name for name in BB_test if name.endswith("txt")])

In [7]:
def original_phrase(phrase):
    return phrase

In [8]:
def stem_phrase(phrase):
    #print(type(phrase))
    #print(phrase)
    
    phrase = phrase.split(" ")
    for i in range(len(phrase)):
        phrase[i] = ps.stem(phrase[i])
    phrase = " ".join(phrase)
    return phrase

In [9]:
def lemmatize_phrase(phrase):
    #print(type(phrase))
    #print(phrase)
    
    phrase = phrase.split(" ")
    for i in range(len(phrase)):
        phrase[i] = wnl.lemmatize(phrase[i])
    phrase = " ".join(phrase)
    return phrase

In [10]:
phrase_reducer = lemmatize_phrase

In [11]:
def get_headword(phrase):
    #print(phrase)
    if phrase == phrase.upper():
        return ""
    
    parses = dep_parser.parse(phrase.split())
    triples = [[(governor, dep, dependent) for governor, dep, dependent in parse.triples()] for parse in parses]
    
    try:
        if(len(triples) > 0 and len(triples[0]) > 0 and len(triples[0][0]) > 0 and len(triples[0][0][0]) > 0 and triples[0][0][0][0] is not None):
            return triples[0][0][0][0]
        else:
            return phrase
    except:
        return phrase
    
    

In [52]:
'asdsa-asdas asda-asda'.replace('-', ' ')

'asdsa asdas asda asda'

In [76]:
def add_documents(txt, processor):
    document = []
    txt = processor(txt)
    sentence_list = sent_tokenize(txt)
    
    for sentence in sentence_list:
        sentence = sentence.replace('-', ' ')
        word_list = word_tokenize(sentence)
        word_list = [word for word in word_list if word not in stopwords.words("english") and word.isalnum()]
        for i in range(len(word_list)):
            if word_list[i].isnumeric():
                word_list[i] = "##number##"
            if word_list[i] != word_list[i].upper():
                word_list[i] = processor(word_list[i].lower())
             
        document.append(word_list)
    #print(document)
    return document

In [77]:
documents = []
for i in range(len(BB_train_txt)):
    txt = open(train_path + BB_train_txt[i],encoding="utf8").read()
    documents += add_documents(txt, phrase_reducer)
for i in range(len(BB_dev_txt)):
    txt = open(dev_path + BB_dev_txt[i],encoding="utf8").read()
    documents += add_documents(txt, phrase_reducer)
for i in range(len(BB_test_txt)):
    txt = open(test_path + BB_test_txt[i],encoding="utf8").read()
    documents += add_documents(txt, phrase_reducer)

In [78]:
model = Word2Vec(documents, min_count=1)

In [79]:
phrase_reducer('Nares')

'Nares'

In [81]:
model['naris']

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 5.0364309e-03, -2.4632025e-03,  2.7730644e-03,  4.3782056e-03,
        2.4468913e-03,  4.5703822e-03, -3.8087831e-03, -2.5896020e-03,
        4.5093182e-03,  2.4804568e-03, -3.3911646e-03, -8.8427641e-04,
        1.0462289e-03,  3.2344898e-03, -2.6480032e-03,  6.6534936e-04,
        3.8329584e-03, -4.0309617e-04, -2.7738274e-03,  3.6357155e-03,
        4.3899368e-04,  2.0577111e-03, -3.5029908e-03, -4.1678236e-03,
       -4.5622126e-03, -3.3995649e-03,  4.2587295e-03,  5.4092193e-03,
        3.1640099e-03, -3.1564683e-03, -1.6722117e-03, -1.7076830e-03,
       -1.7477002e-04,  8.8086503e-04, -9.1277750e-04, -1.7328123e-03,
       -1.0956296e-03,  1.5168773e-03, -1.6244933e-03, -2.6673013e-03,
       -4.6480340e-03, -4.0092198e-03,  1.2935974e-03, -3.1493434e-03,
        2.7778789e-03, -1.6641000e-03,  5.1516463e-04, -8.4838313e-05,
       -2.8898302e-03, -3.2032232e-03,  4.0233233e-03,  9.5485291e-04,
        2.8806494e-03,  1.5260291e-03, -3.9163548e-03,  1.4563302e-03,
      

In [82]:
len(model.wv.vocab)

5537

In [83]:
type(list(model.wv.vocab))

list

In [84]:
len(model[list(model.wv.vocab)[0]])

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


100

In [85]:
def get_phrase_embedding(phrase, model, processor):
    original_phrase = phrase
    phrase = phrase.replace('-', ' ')
    phrase = nltk.word_tokenize(phrase)
    
    phrase = [processor(word) for word in phrase if word.lower() not in stopwords.words("english") and word.isalnum()]
    for i in range(len(phrase)):
        if phrase[i].isnumeric():
            phrase[i] = "##number##"
    
    phrase_size = len(phrase)
    vec_size = len(model[list(model.wv.vocab)[0]])
    embedded_sum = np.zeros(vec_size)
    if(phrase_size == 0):
        return -1
    for word in phrase:
        if word not in list(model.wv.vocab):
            return -1
        embedded_sum = embedded_sum + np.array(model[word])
    embedded_sum = embedded_sum / phrase_size
    
    return embedded_sum
    

In [86]:
def generate_habitat_map(processor, model):
    habitat_list = ontology.split("\n\n[Term]\n")[1:]
    habitat_map = {}
    habitat_map_originals = {}
    habitat_embeddings = {}
    for h in habitat_list:
        #new_habitat = Habitat()
        h = h.split("\n")
        oid = re.findall(r"(?<=id: OBT:)[0-9]+", h[0])[0]
        name = (re.findall(r"(?<=name: ).+", h[1])[0])
        original_name = name
        if name != name.upper():
            name = name.lower()
            name = processor(name)
    
        habitat_map[name] = oid
        habitat_map_originals[oid] = name
        embedding = get_phrase_embedding(name, model, processor)
        if type(embedding) == type(np.array([1])):
            habitat_embeddings[name] = embedding
    
        # word2vec buraya eklenmeli 
        for h_line in h:
            if "synonym" in h_line:
                synonym_name = (re.findall(r"(?<=synonym: \").+(?=\")", h_line)[0])
                if synonym_name != synonym_name.upper():
                    synonym_name = synonym_name.lower()
                    synonym_name = processor(synonym_name)
                if "EXACT" in h_line and synonym_name not in habitat_map:
                    habitat_map[synonym_name] = oid
                    if type(embedding) == type(np.array([1])):
                        habitat_embeddings[synonym_name] = embedding
                    
    return habitat_map, habitat_map_originals, habitat_embeddings

In [99]:
habitat_map, habitat_map_originals, habitat_embeddings = generate_habitat_map(phrase_reducer, model)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [100]:
len(habitat_map)

4000

In [101]:
len(habitat_embeddings)

938

In [102]:
def add_habitats(a1, a2, processor, model):
    a2 = a2.split("\n")
    a1 = a1.split("\n")
    
    
    for annotation in a2:
        a1_line = 0
        t = ""
        oid = ""
        #print(annotation)
        if "OntoBiotope" in annotation:
            t = re.findall(r"(?<=Annotation:)T[0-9]+", annotation)[0]
            oid = re.findall(r"(?<=Referent:OBT:)[0-9]+", annotation)[0]
            named_entity = ""
            
            while t not in a1[a1_line] and a1_line < len(a1)-1:
                a1_line += 1
            if "Habitat" in a1[a1_line]: 
                named_entity = (a1[a1_line].split("\t")[2])
                named_entity = named_entity.replace('-', ' ')
                if named_entity != named_entity.upper():
                    named_entity = named_entity.lower()
                    named_entity = processor(named_entity)
                if named_entity not in habitat_map:
                    habitat_map[named_entity] = oid
                
                if habitat_map_originals[oid] not in habitat_embeddings:
                    habitat_embeddings[habitat_map_originals[oid]] = []
                
                if type(habitat_embeddings[habitat_map_originals[oid]]) == list:
                    emb = get_phrase_embedding(named_entity, model, processor)

                    habitat_embeddings[habitat_map_originals[oid]].append(emb)

In [103]:
for i in range(len(BB_train_txt)):    
    a1 = open(train_path + BB_train_a1[i],encoding="utf8").read()
    a2 = open(train_path + BB_train_a2[i],encoding="utf8").read()
    add_habitats(a1, a2, phrase_reducer, model)

./BioNLP-OST-2019_BB-norm_train/BB-norm-1016123.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-10492485.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-10658649.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-10738994.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-11162736.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-11410343.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-11437594.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-12728302.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-12781527.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-12970344.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-14645268.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-15358511.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-16263187.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-16432479.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-16436701.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-16990433.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-17237163.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-17687514.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-18687046.a1
./BioNLP-OST-

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


./BioNLP-OST-2019_BB-norm_train/BB-norm-8358765.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-8532424.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-8607503.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-8997164.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-9255900.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-9526514.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-9535771.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-9564489.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-9643457.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-9693738.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-9798026.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-9864452.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-F-18524407-000.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-F-18524407-001.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-F-22177851-000.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-F-22177851-001.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-F-22177851-002.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-F-22177851-003.a1
./BioNLP-OST-2019_BB-norm_train/BB-norm-

In [104]:
for key in habitat_embeddings:
    if type(habitat_embeddings[key]) == list:
        vec_size = len(habitat_embeddings[key][0])
        temp_vec = np.zeros(vec_size)
        for vec in habitat_embeddings[key]:
            temp_vec += vec
        temp_vec /= len(habitat_embeddings[key])
        habitat_embeddings[key] = temp_vec

In [107]:
len(habitat_map_originals)

3602

In [106]:
habitat_embeddings

{'fermented cheese': array([ 0.05918138,  0.06826517, -0.02230855,  0.03367625,  0.06700695,
         0.10146591,  0.00037519,  0.12553256, -0.02401156, -0.06327369,
        -0.05003469,  0.07921911, -0.04188737, -0.03276712,  0.02329135,
         0.07768828,  0.12118764, -0.03606056, -0.13050101, -0.09977645,
         0.00693917, -0.0320511 ,  0.10089191,  0.05177309,  0.01967518,
         0.11994801,  0.00691273,  0.15164459,  0.11275943,  0.07799746,
        -0.15652852, -0.05509979,  0.03699312,  0.08446166, -0.09170818,
         0.06774545,  0.09919518,  0.02279173,  0.04453671,  0.0787717 ,
         0.04643207,  0.22112213, -0.00365327,  0.05173734, -0.01566478,
         0.03791698, -0.01303005,  0.0524463 ,  0.01399363, -0.07739837,
        -0.00568215, -0.05751134,  0.0239395 ,  0.06923965,  0.09854987,
        -0.10771048, -0.06465683, -0.00195735, -0.0767545 ,  0.11389613,
         0.04737797,  0.10177546, -0.0033064 ,  0.00934617, -0.01979691,
         0.03240879,  0.0333296

In [93]:
def predict_habitats_exact_matching(a1, txt, a1_name, processor, pred_file_name):
    pred_file = open(pred_file_name, "w")
    cands = 0
    matches = 0
    match_list = {}
    a1 = a1.split("\n")
    count = 1
    for line in a1:
        found = False
        if "Habitat" in line:
            cands += 1
            name = (line.split("\t")[2])
            named_entity = name
            if name != name.upper():
                name = name.lower()
                named_entity = processor(name)
            if named_entity in habitat_map:
                matches += 1
                match = name + " - " + habitat_map[named_entity] + "---exact"
                match_list[line.split("\t")[0]] = [habitat_map[named_entity], named_entity, "exact"]
                found = True
            else:
                #print(name)
                pre_named_entity = get_headword(name)
                #print(pre_named_entity)
                named_entity = processor(pre_named_entity)
                if named_entity in habitat_map:
                    matches += 1
                    match = name + " - " + habitat_map[named_entity] + "---headwordexact"
                    print(match)
                    match_list[line.split("\t")[0]] = [habitat_map[named_entity], name, "headwordexact", named_entity]
                    found = True
        if found:
            pred_file.write("N" + str(count) +"\tOntoBiotope Annotation:" + line.split("\t")[0] + " Referent:OBT:" + match_list[line.split("\t")[0]][0] + "\n")
            count += 1
    #print(a1_name)
    #print("Out of", cands, "candidates,", matches, "matches found")
    #print()
    pred_file.close()
    return match_list

In [ ]:
len(documents)

In [ ]:
def predict_habitats(a1, txt, a1_name, processor, pred_file_name, model):

In [ ]:
annotations_test = {}
for i in range(len(BB_dev_txt)):
    a1 = open(dev_path + BB_dev_a1[i],encoding="utf8").read()
    txt = open(dev_path + BB_dev_txt[i],encoding="utf8").read()
    pred_file_name = "./dev_preds/" + BB_dev_a2[i]
    annotations_test[BB_dev_a1[i]] = predict_habitats(a1, txt, BB_dev_a1[i], phrase_reducer, pred_file_name)

In [ ]:
annotations_test

In [ ]:
def test_habitats(a1, a2, processor):
    
    a2 = a2.split("\n")
    a1 = a1.split("\n")
    match_list = {}
    for annotation in a2:
        a1_line = 0
        t = ""
        oid = ""
        #print(annotation)
        if "OntoBiotope" in annotation:
            t = re.findall(r"(?<=Annotation:)T[0-9]+", annotation)[0]
            oid = re.findall(r"(?<=Referent:OBT:)[0-9]+", annotation)[0]
            named_entity = ""
            while t not in a1[a1_line] and a1_line < len(a1)-1:
                a1_line += 1
            if "Habitat" in a1[a1_line]:
                named_entity = (a1[a1_line].split("\t")[2])
                if named_entity != named_entity.upper():
                    named_entity = named_entity.lower()
                    named_entity = processor(named_entity)
                match_list[t] = [oid, named_entity]
    return match_list

In [ ]:
dev_a2 = {}
for i in range(len(BB_dev_txt)):
    a1 = open(dev_path + BB_dev_a1[i],encoding="utf8").read()
    a2 = open(dev_path + BB_dev_a2[i],encoding="utf8").read()
    dev_a2[BB_dev_a1[i]] = test_habitats(a1,a2, phrase_reducer)

In [ ]:
true_pred_count = 0
false_pred_count = 0
total_count = 0
for key in dev_a2:
    """print()
    print(key)
    print(dev_a2[key])
    print("**************")"""
    total_count += len(dev_a2[key])
    if key in annotations_test:
        for pred in annotations_test[key]:            
            if annotations_test[key][pred][0] == dev_a2[key][pred][0]:
                true_pred_count += 1
            else:
                print(key)
                print("Prediction:", annotations_test[key][pred]," Real:", dev_a2[key][pred])
                print("Prediction:", habitat_map_originals[annotations_test[key][pred][0]], " Real:", habitat_map_originals[dev_a2[key][pred][0]])
                false_pred_count += 1

In [ ]:
recall = true_pred_count/total_count
recall

In [ ]:
precision = true_pred_count/(true_pred_count + false_pred_count)
precision

## Denemeler